Niranjan Jangir
B21CH021

Importing necessary libraries:

In [9]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

Reading the matlab dataset

In [10]:
training_data = loadmat(r"Training_Data_Four_Tanks.mat")
testing_data = loadmat(r"Testing_Data_Four_Tanks.mat")

Reading this training and testing MATLAB dataset as pandas dataframe along with extracting it's input, output and time span data separately.

In [11]:
train_input_data = pd.DataFrame(training_data['input_data'],columns = ['v1','v2'])  #input features
train_output_data = pd.DataFrame(training_data['output_data'],columns =['h1','h2']) #output features
train_time_span = pd.DataFrame(training_data['time_span'],columns=['time']) #time span

In [12]:
test_input_data = pd.DataFrame(testing_data['input_data'],columns = ['v1','v2']) #input features
test_output_data = pd.DataFrame(testing_data['output_data'],columns =['h1','h2']) #output features
test_time_span = pd.DataFrame(testing_data['time_span'],columns=['time']) #time span

To use LSTM, we convert the input data to first a numpy array, then reshape this array as below:

In [13]:
train_input_data_array = train_input_data.to_numpy()
train_input_data_lstm = train_input_data_array.reshape(train_input_data_array.shape[0], 1, train_input_data_array.shape[1])

Here is the implementation of a LSTM model:

In [27]:
# defining a function to create the model:
def get_lstm_model(n_inputs, n_outputs):
    model = Sequential()  # Initialize a Sequential model
    model.add(LSTM(64, input_shape=(None, n_inputs), kernel_initializer='he_uniform', activation='relu'))  # Add an LSTM layer with 64 units, ReLU activation, and He uniform weight initialization
    model.add(BatchNormalization())  # Add batch normalization layer to stabilize and accelerate the learning process
    model.add(Dropout(0.2))  # Add dropout layer to randomly set a fraction of input units to 0 at each update during training to reduce overfitting
    model.add(Dense(32, activation='relu'))  # Add a fully connected layer with 32 units and ReLU activation
    model.add(Dense(n_outputs))  # Add the output layer
    return model  # Return the model

# Define and compile the LSTM model on the input training data
lstm_model = get_lstm_model(train_input_data.shape[1], train_output_data.shape[1])
optimizer = Adam(learning_rate=0.001)  # Custom learning rate for optimization
lstm_model.compile(loss='mae', optimizer=optimizer) #compiling the model

# Training the LSTM model
lstm_model.fit(train_input_data_lstm, train_output_data, verbose=1, epochs=30, validation_split=0.2)

Epoch 1/30
2250/2250 [==============================] - 13s 5ms/step - loss: 1.5579 - val_loss: 0.4431
Epoch 2/30
2250/2250 [==============================] - 10s 4ms/step - loss: 0.6456 - val_loss: 0.4640
Epoch 3/30
2250/2250 [==============================] - 13s 6ms/step - loss: 0.4437 - val_loss: 0.5215
Epoch 4/30
2250/2250 [==============================] - 11s 5ms/step - loss: 0.2607 - val_loss: 0.5776
Epoch 5/30
2250/2250 [==============================] - 11s 5ms/step - loss: 0.2156 - val_loss: 0.5663
Epoch 6/30
2250/2250 [==============================] - 12s 5ms/step - loss: 0.2066 - val_loss: 0.5561
Epoch 7/30
2250/2250 [==============================] - 10s 5ms/step - loss: 0.2129 - val_loss: 0.5797
Epoch 8/30
2250/2250 [==============================] - 9s 4ms/step - loss: 0.2020 - val_loss: 0.5898
Epoch 9/30
2250/2250 [==============================] - 10s 5ms/step - loss: 0.1975 - val_loss: 0.5506
Epoch 10/30
2250/2250 [==============================] - 10s 4ms/step - lo

Testing the implementated model on the testing dataset and printing the loss:

In [28]:
# Convert data to numpy array and reshape for LSTM
test_input_data_array = test_input_data.to_numpy()
test_input_data_lstm = test_input_data_array.reshape(test_input_data_array.shape[0], 1, test_input_data_array.shape[1])

# Evaluate the LSTM model on test data
loss = lstm_model.evaluate(test_input_data_lstm, test_output_data)
print("Test Loss:", loss)


1876/1876 [==============================] - 4s 2ms/step - loss: 0.3672
Test Loss: 0.3672342300415039


Conclusions:
1. LSTM took more time to compile for same number of epochs as compared to RNN due to the increased complexity of LSTM layers, which involve more computations and parameters to train, such as memory cells and gates.
2. when epochs = 100 : loss = 0.3439,
when epochs = 80 : loss = 0.3011,
when epochs = 60 : loss = 0.37
3. We can see that with optimal epochs, LSTM has slightly less accuracy than RNN. It may be due to more complexity and more parameters, which can lead to overfitting or difficulty in capturing patterns in the data effectively, resulting in higher loss.